In [ ]:
pip install youtube-transcript-api

In [ ]:
pip install --upgrade google-api-python-client

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import SRTFormatter
from googleapiclient.discovery import build
import os
import re

In [ ]:
# Initialize the YouTube API client
api_key = os.getenv("YOUTUBE_API_KEY")

# Initialize the YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

def get_video_ids_with_captions(query, max_results=50):
    video_ids = []
    # Make a search request to the YouTube API
    search_response = youtube.search().list(
        q=query,
        part='id',
        maxResults=max_results,
        type='video',
        videoCaption='closedCaption',  # Only retrieve videos with captions
    ).execute()

    # Extract video IDs
    for search_result in search_response.get('items', []):
        video_ids.append(search_result['id']['videoId'])

    return video_ids

def sanitize_filename(filename):
    """Sanitize the filename by removing or replacing invalid characters."""
    valid_filename = re.sub(r'[^\w\s-]', '', filename).strip().lower()
    return re.sub(r'[-\s]+', '-', valid_filename)

def main():
    query = "hindi old comedy movies"  # Update this with your search term
    video_ids = get_video_ids_with_captions(query)

    # Ensure the directory exists
    output_dir = "video_ids"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Sanitize the query to create a valid filename
    filename = os.path.join(output_dir, f"{sanitize_filename(query)}.txt")

    # Save the video IDs to a file named after the query, inside the specified directory
    with open(filename, "w") as file:
        for video_id in video_ids:
            file.write(f"{video_id}\n")

    print(f"Saved {len(video_ids)} video IDs to {filename}.")

if __name__ == "__main__":
    main()

In [4]:
def download_transcript(video_id, output_folder):
    try:
        # Fetching the transcript in Hindi
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
        
        # Formatting the transcript as SRT
        formatter = SRTFormatter()
        srt_transcript = formatter.format_transcript(transcript)

        # Ensure the output directory exists
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Saving the transcript as an SRT file
        filename = os.path.join(output_folder, f"{video_id}.srt")
        with open(filename, "w") as text_file:
            text_file.write(srt_transcript)
            
        print(f"Transcript for video {video_id} has been saved in '{output_folder}' folder.")
    except Exception as e:
        print(f"An error occurred with video {video_id}: {e}")

def process_video_ids(directory, output_folder):
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r') as file:
                video_ids = file.read().splitlines()
                for video_id in video_ids:
                    download_transcript(video_id, output_folder)

if __name__ == "__main__":
    video_ids_directory = "video_ids"
    subtitles_output_folder = "subtitles"
    process_video_ids(video_ids_directory, subtitles_output_folder)

Transcript for video bVOotlaiYhg has been saved in 'subtitles' folder.
Transcript for video z7MIyQS9p5E has been saved in 'subtitles' folder.
Transcript for video cuwRniQ1-Y8 has been saved in 'subtitles' folder.
Transcript for video iycEzub4oNE has been saved in 'subtitles' folder.
Transcript for video 67kSNheqKX4 has been saved in 'subtitles' folder.
Transcript for video 8Rs88dmsAqA has been saved in 'subtitles' folder.
An error occurred with video CucUebxNoNA: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=CucUebxNoNA! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (CucUebxNoNA) transcripts are available in the following languages:

(MANUALLY CREATED)
 - hi-Latn ("Hindi (Latin)")[TRANSLATABLE]

(GENERATED)
None

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")


Transcript for video W38_qfRrz6Q has been saved in 'subtitles' folder.
An error occurred with video 6InLqsINcWw: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=6InLqsINcWw! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (6InLqsINcWw) transcripts are available in the following languages:

(MANUALLY CREATED)
 - hi ("Hindi")[TRANSLATABLE]

(GENERATED)
None

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("Danish")
 - dv ("Divehi")
 - nl ("Dutch")
 

Transcript for video EI2P2egaoF0 has been saved in 'subtitles' folder.
An error occurred with video tJBaETMtksc: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=tJBaETMtksc! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (tJBaETMtksc) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-GB ("English (United Kingdom)")[TRANSLATABLE]

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - 

Transcript for video GAjNnP5VN5c has been saved in 'subtitles' folder.
An error occurred with video 94FLE3kxIzk: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=94FLE3kxIzk! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (94FLE3kxIzk) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-IN ("English (India)")[TRANSLATABLE]

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czec

An error occurred with video ZeNjr6yB_Ow: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=ZeNjr6yB_Ow! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (ZeNjr6yB_Ow) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-GB ("English (United Kingdom)")[TRANSLATABLE]

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("Danish")
 - dv ("Divehi")
 - nl ("Dutch")
 - en ("

...

Transcript for video 8dQTdJszjO8 has been saved in 'subtitles' folder.
An error occurred with video tWAZE48d8N4: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=tWAZE48d8N4! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (tWAZE48d8N4) transcripts are available in the following languages:

(MANUALLY CREATED)
 - hi ("Hindi")[TRANSLATABLE]

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("D

An error occurred with video PpcXOCKGLQ8: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=PpcXOCKGLQ8! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (PpcXOCKGLQ8) transcripts are available in the following languages:

(MANUALLY CREATED)
 - en-IN ("English (India)")[TRANSLATABLE]

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("Danish")
 - dv ("Divehi")
 - nl ("Dutch")
 - en ("English")

Transcript for video jdsnC3SJzZs has been saved in 'subtitles' folder.
An error occurred with video 4dfujZgzJho: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=4dfujZgzJho! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (4dfujZgzJho) transcripts are available in the following languages:

(MANUALLY CREATED)
 - hi ("Hindi")[TRANSLATABLE]

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("D

An error occurred with video rhg4dx44aVM: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=rhg4dx44aVM! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (rhg4dx44aVM) transcripts are available in the following languages:

(MANUALLY CREATED)
 - hi ("Hindi")[TRANSLATABLE]

(GENERATED)
 - hi ("Hindi (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("Danish")
 - dv ("Divehi")
 - nl ("Dutch")
 - en ("English")
 - eo ("Espe

Transcript for video Rri83pDN6kc has been saved in 'subtitles' folder.
